In [16]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [2]:
# The first tool is wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200))

In [14]:
# Tool number two will be a retriever tool for my personal site
# first I load the data
loader = WebBaseLoader("https://francoismavunila.github.io/")
docs = loader.load()

# then split it into smaller chunks
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

# then create embeddings and save it on the vector store
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
# then I can create a retriever
retriever=vectordb.as_retriever()
# then create a retriever tool
retriever_tool = create_retriever_tool(retriever, "francois_mavunila", "search for information about Francois Mavunila, for any information about Francois Mavunila use this tool")

In [17]:
# Tool number 3 - Arxiv Tool - an open archive for scholarly articles
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [19]:
tools = [wikipedia, arxiv, retriever_tool]